In [ ]:
import os

from LandcoverDataset import LandcoverDataset as LD
from DataVis import *
from Model import CNN, CNN_load

**Sentinel-2A**
|**N**        |0         |1          |2        |3     |4        | 5 | 6 | 7 | 8 | 9 | 10|
|-------------|----------|-----------|---------|------|---------|---|---|---|---|---|---|
|**channel**  |          |B02        |B03      |B04   |B05      |B06|B07|B08|B8A|B11|B12|
|**seg. mask**|open water|settlements|bair soil|forest|grassland|   |   |   |   |   |   |


# Reading Data and creating Dataloader example in Pytorch

In [ ]:
TRAIN_PATH = "datasets/USA/train/"
IMG_TRAIN = TRAIN_PATH + "/images/"
MASK_TRAIN = TRAIN_PATH + "/masks/"
l_train = len(LD.filenames_in_dir(IMG_TRAIN))

VALID_PATH = "datasets/USA/valid/"
IMG_VALID = VALID_PATH + "/images/"
MASK_VALID = VALID_PATH + "/masks/"
l_valid = len(LD.filenames_in_dir(IMG_VALID))

In [ ]:
print(f"Train samples: {l_train}")
print(f"Test samples: {l_valid}")

In [ ]:
BATCH = 4

ds_train = LD(
    img_path = IMG_TRAIN, 
    mask_path= MASK_TRAIN,
    n1 = 0,
    n2 = l_train
)
dl_train = ds_train.load_data(BATCH)

ds_valid = LD(
    img_path=IMG_VALID, 
    mask_path=MASK_VALID,
    n1 = 0,
    n2 = l_valid
)
dl_valid = ds_valid.load_data(BATCH)

In [ ]:
print(f"Images: {ds_train[0][0].shape}")
print(f"Masks:  {ds_train[0][1].shape}")

In [ ]:
peek_dataset(ds_train)

In [ ]:
input_channels = 10
hidden_layers = [16, 16]
model = CNN(input_channels, hidden_layers)
model.model

In [ ]:
num_epochs = 10

In [ ]:
learning_rate = 0.002
epochs_sum = 0

In [ ]:
model.train(dl_train, dl_valid, num_epochs, learning_rate)
epochs_sum += num_epochs

In [ ]:
img, mask = ds_train.rand_samp_names()
prediction = model.predict(img)
compare_prediction(img, mask, prediction)

In [ ]:
m_name = f"ne={epochs_sum}, hl={hidden_layers}"
model.pickle(f"models/{m_name}.torch")

In [ ]:
model.unpickle("models/model.torch")

In [ ]:
model = CNN_load("models/model.torch")